In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:  
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression , Ridge , LogisticRegression
from sklearn.preprocessing import LabelEncoder , PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score , mean_squared_error
from scipy import stats
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from graphviz import Source
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix , classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')
sns.set(style="darkgrid")

plt.style.use('fivethirtyeight')

In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [4]:
from sklearn import preprocessing


In [5]:
train = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/train.csv", index_col=None)
test = pd.read_csv("/kaggle/input/airline-passenger-satisfaction/test.csv", index_col=None)

In [6]:
def dataset_overview(data, col):
    
    print("------------")
    #print(data.head())
    print("-----------")
    print("---------------")

    print(data.columns)
    print("------------")
    print("---------------")

    print("Shape of the dataset")
    print(data.shape)
    print("-------------")
    print("---------------")

    print("Null Value counts")
    print(data.isnull().sum())
    print("-------------")
    print("---------------")

    print("dataset informaation")
    print(data.info())
    print("---------------")
    print("---------------")
    
    print("The outcome values",data[col].value_counts(normalize=True))
    plt.figure(figsize=(10,5))
    print(sns.countplot(x=col, data=data))
    plt.show()

In [7]:
dataset_overview(train, 'satisfaction')

In [8]:
train=train.drop('Unnamed: 0', axis=1)

In [9]:
test=test.drop('Unnamed: 0', axis=1)

In [10]:
train=train.fillna(train.mean())

In [11]:
test=test.fillna(test.mean())

In [12]:
for _ in train.columns:
    print("The number of null values in:{} == {}".format(_, train[_].isnull().sum()))

In [13]:
for _ in test.columns:
    print("The number of null values in:{} == {}".format(_, test[_].isnull().sum()))

In [14]:
#from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings("ignore")


In [15]:
def correlation_matrix(data):
    corr = data.corr().round(2)

    # Mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set figure size
    f, ax = plt.subplots(figsize=(20, 20))

    # Define custom colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap
    d=sns.heatmap(corr, mask=mask, cmap=cmap, vmin=-1, vmax=1, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

    plt.tight_layout()
    return d

In [16]:
correlation_matrix(train)

In [17]:
from sklearn.feature_selection import RFE, SelectFromModel

In [18]:
categorical_features=['Gender', 'Customer Type', 'Type of Travel', 'Class']
categorical_transformer = Pipeline(
    [
        ('imputer_cat', SimpleImputer(strategy = 'constant', fill_value = 'missing')),
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

In [19]:
numeric_features=['id', 'Age',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

numeric_transformer = Pipeline(
    [
        ('imputer_num', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler())
    ]
)

In [20]:
preprocessor = ColumnTransformer(
    [
        ('categoricals', categorical_transformer, categorical_features),
        ('numericals', numeric_transformer, numeric_features)
    ],
    remainder = 'drop'
)

In [21]:
# Create the pipeline, with the tranformer and the model
pipeline = Pipeline(
    [
        ('preprocessing', preprocessor),
        ("features_selection",RFE(LogisticRegression(), step = 1)),
        ('clf', XGBClassifier(alpha=1.5))
    ]
)

In [22]:
# Create the parameters for the different models
params = [
    {'features_selection':[RFE(XGBClassifier(n_estimator=50, max_depth=9,  alpha=1.5)), 
           SelectFromModel(XGBClassifier(n_estimator=50, max_depth=9, alpha=1.5)),
           None],
    'clf__n_estimators': np.arange(50, 750, 50),
     'clf__max_depth': np.arange(2, 10, 1),
     'clf__lambda': [0.5,0.75,1,1.5],
     'clf__alpha': [0,0.5,0.75,1,1.5],
    }
]

In [23]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV

In [24]:
cv = KFold(2, shuffle=True, random_state=42)

In [25]:
warnings.filterwarnings('ignore')
# Create the random search
ramd_grid = RandomizedSearchCV(pipeline, params, scoring="f1", n_jobs=-1, cv=cv, n_iter=200, verbose=1, random_state=42)


In [26]:
def label_encoding(data, col):
    label_encoder = preprocessing.LabelEncoder()
    data[col]=label_encoder.fit_transform(data[col])
    return data[col].head()

In [27]:
label_encoding(train, 'satisfaction')

In [28]:
X=train.drop('satisfaction', axis=1)
y=train.satisfaction


In [ ]:
ramd_grid.fit(X,y)

In [ ]:
ramd_grid_rf.best_params_

In [ ]:
# Get the performance of the model in the train set
y_pred_train_rf = ramd_grid_rf.predict(X)
print(classification_report(y, y_pred_train_rf))

# For test data set

In [1]:
label_encoding(test, 'satisfaction')

In [2]:
X_test=test.dropna()
y_test=test.satisfaction

In [3]:
# Get the performance of the model in the train set
y_pred_test_rf = ramd_grid_rf.predict(X_test)
print(classification_report(y_test, y_pred_test_rf))

In [4]:
!pip install dill


In [5]:
import dill as pickle
filename = 'model_v1.pk'

In [6]:
with open('../flask_api/models/'+filename, 'wb') as file:
    pickle.dump(ramd_grid_rf, file)